# HDS5210-2020 Midterm

In the midterm, you're going to focus on using the programming skills that you've developed so far to build a calculator for the Apache II scoring system for ICU Mortality.  
* https://www.mdcalc.com/apache-ii-score#evidence
* https://reference.medscape.com/calculator/apache-ii-scoring-system

For the midterm, we'll be building a calculator for the Apache II score and then running that against a patient file that's available to you out on the internet.  This will be broken down into three main steps:
1. Create your JSON file to encapsulate all of the calculation rules for Apache II
2. Create functions to calculate the Apache II score using your JSON configuration
3. Create a function to loop over the patients in a file on the internet and calculate Apach II scores for all of them



---

## Part 1: Creating a JSON Rules File

Look at the rules for the Apache II scoring system on the pages above.  The first step in the midterm is to use those rules and create a JSON configuration file as described in the 2019 midterm video.  I've provided a starter file named `apache.json` to get you started.

Inside that file, you'll find placeholders for all of the measures that go into the Apache II scoring model:
* Organ Failure History
* Age
* Temperature
* [pH](https://en.wikipedia.org/wiki/PH)
* Heart rate
* Respiratory rate
* [Sodium](https://www.mayoclinic.org/diseases-conditions/hyponatremia/symptoms-causes/syc-20373711)
* [Potassium](https://www.emedicinehealth.com/hyperkalemia/article_em.htm)
* [Creatinine](https://www.medicalnewstoday.com/articles/322380)
* [Hematocrit](https://labtestsonline.org/tests/hematocrit)
* White Blood Count
* [FiO2](https://www.ausmed.com/cpd/articles/oxygen-flow-rate-and-fio2)
* [PaO2](https://www.verywellhealth.com/partial-pressure-of-oyxgen-pa02-914920)
* [A-a gradient](https://www.ncbi.nlm.nih.gov/books/NBK545153/)


You may need to create a sort of nested set of rules in some cases.  For instance, the rule for Creatinine says to use certain ranges and points in the case of Acute Renal Failure and a different set of points for Chronic Renal Failure.

Similarly, the rule for FiO2 says to use PaO2 to calculate scores if the FiO2 is <50, and to use A-a Gradient if the PaO2 is >50.

When you've created your `apache.json` file, make sure it's in the same directory as this notebook.

### Testing your JSON

The assert() functions below should all run just fine.  If you want to change the names of any of the keys in the JSON I provided you, you may, but you'll also need to update this test code so that it doesn't fail.  Remember, your notebook should be able to run end-to-end before you submit it.

In [562]:
import json

with open('apache.json') as f:
    rules = json.load(f)

assert('Organ Failure History' in rules.keys())
assert('Age' in rules.keys())
assert('Temperature' in rules.keys())
assert('pH' in rules.keys())
assert('Heart Rate' in rules.keys())
assert('Respiratory Rate' in rules.keys())
assert('Sodium' in rules.keys())
assert('Potassium' in rules.keys())
assert('Creatinine' in rules.keys())
assert('Hematocrit' in rules.keys())
assert('White Blood Count' in rules.keys())
assert('FiO2' in rules.keys())

---

## Part 2: Functions to evaluate rules

Write a series of functions, enough to satisfy all of the main criteria that we're using to calculate the Apache II score.  That list is the same as the assert statements above.

* Each of your functions should be well documented.
* Each function should have "config_file" as one of it's parameters.
* Each function should return a numerical score value.
* Similar to what we discussed in the review, if you can generalize some rules, do so.  You should **NOT** end up with one function for each input variable.  If you did that, you'd have a lot of repetative code.

The Glasgow Coma Scale is simply a 1-to-1 score translation.  Simply add the Glasgow Coma Scale value.  So, you don't need to write a function for this. [Glasgow Coma Scale](https://www.cdc.gov/masstrauma/resources/gcs.pdf)

**CORRECTION ADDED 2/29** - The Glasgow Coma Scale points should be calculated as `1 - Glasgow Coma Scale` rather than what I just stated above.  My preference would be that you do the calculation correctly, as per MDCalc, and then use the **corrected** scores files to compare against as noted in Part 4.

### Testing you Functions

Write enough test cases to verify that your functions work for evaulating all of the scoring inputs.  Have at least 3 test cases for each input.

These tests can be written the same as the assertions we've use in previous assignments.  For example, if you a function for `temperature_score` then you write a test case like:

```
assert( temperature_score(37) == 0 )
```

In [563]:
import json 

def organ_failure_score(history_list, config_file):
    """ list, str -> int
    history_list: A list of strings that includes if this subject has had a history of severe organ insufficiency 
    or immunocompromised
    config_file: The name of a configuration file to use for scoring this history 
    
    The function should return a score for this subject based on the history of the subject in the history_list
    and the scoring rules in the config_file.  For instance, if the subject has had a history of severe organ insufficiency 
    and was nonoperative or emergency postoperative patient then the score would be 5 and the function should return 5 
    """
       
    score = 0
    
    ### BEGIN SOLUTION
    
    config = json.load(open(config_file))
    organ_failure_scores = config.get("Organ Failure History")
    
    for history in history_list:
        if history in organ_failure_scores:
            score += organ_failure_scores.get(history)
    
    ### END SOLUTION

    return score

In [564]:
CONFIG_FILE = "apache.json"

assert(organ_failure_score([],CONFIG_FILE) == 0)
assert(organ_failure_score(['Nonoperative'],CONFIG_FILE) == 5)
assert(organ_failure_score(['Elective'],CONFIG_FILE) == 2)
assert(organ_failure_score(['Emergency'],CONFIG_FILE) == 5)

In [565]:
import json 

def mortality_score(value, config_file, variable_name):
    """ int, str -> int
    rules: measurement as an integer value for variables of Age, Temperature, pH, Heart Rate, 
    Respiratory Rate, Sodium level, Potassium level, Hematocrit level, and White blood cell count.  
    config_file: The name of a configuration file to use for scoring the ICU mortality risk
    
    The function should return a score for this subject based on where their actual variables
    fall in the scoring rules.
    """
    score = 0
    
 ### BEGIN SOLUTION
    
    config = json.load(open(config_file))
    rules = config.get(variable_name)
    
    for rule in rules:
        if value >= rule.get('min') and value < rule.get('max'):
            score = rule.get('points')

 ### END SOLUTION
    
    return score


In [566]:
CONFIG_FILE = "apache.json"

assert(mortality_score(34, CONFIG_FILE, "Age") == 0)
assert(mortality_score(88, CONFIG_FILE, "Age") == 6)
assert(mortality_score(60, CONFIG_FILE, "Age") == 3)

assert(mortality_score(40, CONFIG_FILE, "Temperature") == 3)
assert(mortality_score(43, CONFIG_FILE, "Temperature") == 4)
assert(mortality_score(32, CONFIG_FILE, "Temperature") == 2)
assert(mortality_score(37, CONFIG_FILE, "Temperature") == 0)

assert(mortality_score(7.2, CONFIG_FILE, "pH") == 3)
assert(mortality_score(7.4, CONFIG_FILE, "pH") == 0)
assert(mortality_score(7.7, CONFIG_FILE, "pH") == 4)
assert(mortality_score(7.5, CONFIG_FILE, "pH") == 1)

assert(mortality_score(20, CONFIG_FILE, "Heart Rate") == 4)
assert(mortality_score(50, CONFIG_FILE, "Heart Rate") == 3)
assert(mortality_score(120, CONFIG_FILE, "Heart Rate") == 2)
assert(mortality_score(80, CONFIG_FILE, "Heart Rate") == 0)

assert(mortality_score(2, CONFIG_FILE, "Respiratory Rate") == 4)
assert(mortality_score(20, CONFIG_FILE, "Respiratory Rate") == 0)
assert(mortality_score(40, CONFIG_FILE, "Respiratory Rate") == 3)

assert(mortality_score(110, CONFIG_FILE, "Sodium") == 4)
assert(mortality_score(150, CONFIG_FILE, "Sodium") == 1)
assert(mortality_score(130, CONFIG_FILE, "Sodium") == 0)

assert(mortality_score(3, CONFIG_FILE, "Potassium") == 1)
assert(mortality_score(4, CONFIG_FILE, "Potassium") == 0)
assert(mortality_score(7, CONFIG_FILE, "Potassium") == 4)

assert(mortality_score(15, CONFIG_FILE, "Hematocrit") == 4)
assert(mortality_score(45, CONFIG_FILE, "Hematocrit") == 0)
assert(mortality_score(20, CONFIG_FILE, "Hematocrit") == 2)

assert(mortality_score(15, CONFIG_FILE, "White Blood Count") == 1)
assert(mortality_score(60, CONFIG_FILE, "White Blood Count") == 4)
assert(mortality_score(0.4, CONFIG_FILE, "White Blood Count") == 4)
assert(mortality_score(3, CONFIG_FILE, "White Blood Count") == 0)

In [567]:
import json 

def creatinine_score(value, config_file, disease):
    """ int -> float 
    creatinine_level: A list of strings that includes if a patient has acute renal disease, chronic renal disease, or no renal disease
    and the creatinine levels possible for that patient. 
    config_file: The name of a configuration file to use for scoring the creatinine level
    
    The function should return a score based on the patients creatinine level and if the patient has a history of renal disease or not.
    """
    score = 0
    
 ### BEGIN SOLUTION
    
    config = json.load(open(config_file))
    creatinine_rules = config.get("Creatinine").get(disease)
    
    for rule in creatinine_rules: 
        if value >= rule.get('min') and value < rule.get('max'):
            score = rule.get('points')  

 ### END SOLUTION 
    
    return score
    

In [568]:
CONFIG_FILE = "apache.json"

assert(creatinine_score(3, CONFIG_FILE, "Chronic Renal Failure") == 3)
assert(creatinine_score(1.3, CONFIG_FILE, "Chronic Renal Failure") == 0)
assert(creatinine_score(1.6, CONFIG_FILE, "Chronic Renal Failure") == 2)

assert(creatinine_score(1.5, CONFIG_FILE, "Acute Renal Failure") == 4)
assert(creatinine_score(4, CONFIG_FILE, "Acute Renal Failure") == 8)
assert(creatinine_score(2.4, CONFIG_FILE, "Acute Renal Failure") == 6)
assert(creatinine_score(0.7, CONFIG_FILE, "Acute Renal Failure") == 0)

In [607]:
import json 

def oxgeniation_score(value, PaO2, Aa_gradient, config_file):
    
    score = 0
    
 ### BEGIN SOLUTION
    config = json.load(open(config_file))
    
    if value <50:
        rules = config.get('FiO2')
        rules = rules.get('Pa02')
        for rule in rules:
            if PaO2 >= rule.get('min') and PaO2 <= rule.get('max'):
                score = rule.get('points')
    else:
        rules = config.get('FiO2')
        rules = rules.get('Aa_Gradient')
        for rule in rules:
            if Aa_gradient > rule.get('min') and Aa_gradient < rule.get('max'):
                score = rule.get('points')
    
    
 ### END SOLUTION
    
    return score

In [608]:
CONFIG_FILE = "apache.json"
assert(oxgeniation_score(70, 458, 59, config_file)== 0 )
assert(oxgeniation_score(66, 66, 450, config_file)== 3 )
assert(oxgeniation_score(33, 69, 490, config_file)== 1 )

---

## Part 3: Put it all together

Create a new function called `apache_score()` that takes all of the necessary inputs and returns the final Apache II score.  Use any variable names that you want.  For clarity and organization, my recommendation is to create them in the same order as they're documented in the website.

1. Organ Failure History
2. Age
3. Temperature
4. pH 
5. Heart rate
6. Respiratory rate
7. Sodium
8. Potassium
9. Creatinine
10. Acute renal failure
11. Hematocrit
12. White Blood Count
13. Glasgow Coma Scale
14. FiO2
15. PaO2
16. A-a gradient


### Testing your Function

Write a few test cases to make sure that your code functions correctly.  In the last step, you'll have LOTS of test cases run through, but you should do some of your before moving on.

In [613]:
def glascow_score(glascow_scale):
    glascowscore = 15 - glascow_scale 
    
    return glascowscore

In [618]:
import json 

def apache_score(history_list, age, temperature, pH, heart_rate, respiratory_rate, sodium, potassium, 
                 creatinine, acute_renal, hematocrit, white_blood_count, glasgow_coma_scale, FiO2, PaO2, Aa_gradient):
    """ str, int, -> float
    This function will take all necessary inputs and returns the final Apache II score. 
    
    * Organ_failure_score is a list of strings like like "nonoperative" and "elective"
    * mortality_score contains multiple variables like "Age", "pH", "Heart Rate", "Respiratory Rate",
    "Sodium", "Potassium", "Hematocrit", and "White Blood Count" 
    * creatinine_score is a dictonary that contains acute renal failure and chronic renal failure
    and the creatinine level assoicated with each disease
    * oxgeniation_score is a dictonary that contains the oxgenation level and it will use PaO2 if FiO2 <50% otherwise
    it will use the A-a gradient. 
    """
    
    score = 0 
    
 ### BEGIN SOLUTION 

    config_file = 'apache.json'
    
    score += organ_failure_score(history_list, config_file)
    score += mortality_score(age, config_file, 'Age')
    score += mortality_score(temperature, config_file, 'Temperature')
    score += mortality_score(pH, config_file, 'pH')
    score += mortality_score(heart_rate, config_file, 'Heart Rate')
    score += mortality_score(respiratory_rate, config_file, 'Respiratory Rate')
    score += mortality_score(sodium, config_file, 'Sodium')
    score += mortality_score(potassium, config_file, 'Potassium')
    score += creatinine_score(creatinine, config_file, 'Chronic Renal Failure')
    score += creatinine_score(acute_renal, config_file, 'Acute Renal Failure')
    score += mortality_score(hematocrit, config_file, 'Hematocrit')
    score += mortality_score(white_blood_count, config_file, 'White Blood Count')
    score += glascow_score(glasgow_coma_scale)
    score += oxgeniation_score(FiO2, PaO2, Aa_gradient, config_file)
    
 ### END SOLUTION

    return score 

In [619]:
apache_score("Elective", 45, 32, 7.14, 70, 12, 145, 4.0, 1.4, 4.0, 34, 12, 30, 61, 250, 440)

4

---

## Part 4: Accessing and processing the patient file

Fill out the simple function below to retrieve the patient data as a CSV file from any given URL and return a list of all of the Apache II scores based on the data you find for those patients.
* The patient file will be a CSV
* It will have column headers that match the labels shown above
* The columns will not necessarily appear in the order shown above
* You should output only the Apache II scores, not any other information
* Your output should be a list in the same order as the input rows

In [633]:
import pandas as pd

test_patient = pd.read_csv('https://hds5210-2020.s3.amazonaws.com/TestPatients.csv')

 ### BEGIN SOLUTION 
def test_function(test_patient, config_file):
    check = pd.read_csv(test_patient)
    scores=[]
    for x in check.index: 
        score=(apache_score(check['Organ Failure History'][x], 
              check['Age'][x],
              check['Temperature'][x],
              check['pH'][x],
              check['Heart Rate'][x],
              check['Respiratory Rate'][x],
              check['Sodium'][x],
              check['Potassium'][x],
              check['Creatinine'][x],
              check['Hematocrit'][x],
              check['White Blood Count'][x],
              check['Glasgow Coma Scale'][x],
              check['FiO2'][x],              
              check['PaO2'][x],
              check['A-a Gradient'][x],
              config_file))
        
        scores.append(score)
  
 ### END SOLUTION 
    return scores

In [634]:
test_function('https://hds5210-2020.s3.amazonaws.com/TestPatients.csv', 'apache.json')

TypeError: '>' not supported between instances of 'str' and 'int'

### Testing your Function

The URL for the test data is: https://hds5210-2020.s3.amazonaws.com/TestPatients.csv


You can verify your results by comparing them against this data: https://hds5210-2020.s3.amazonaws.com/Scores.csv

**CORRECTION ADDED 3/29** - If you calculated the Glasgow Coma Scale points as per the actual instructions in MDCalc, then please use this set of corrected scores to compare your results with: https://hds5210-2020.s3.amazonaws.com/Scores_corrected.csv


In [603]:
test_patient_2 = pd.read_csv("https://hds5210-2020.s3.amazonaws.com/Scores_corrected.csv")

scores_2 = []
for index, row in test_patient_2.iterrows():
    scores_2.append(int(row["TOTAL"]))

In [604]:
scores == scores_2

False